In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (SparkSession.builder
         .master("spark://spark-master:7077")
         .appName("read-minio-defaults")
         .getOrCreate())

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /opt/spark/ivy/cache
The jars for the packages stored in: /opt/spark/ivy/jars
org.apache.spark#spark-hadoop-cloud_2.13 added as a dependency
io.delta#delta-spark_2.13 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-4.0_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-27031d66-0b79-49aa-8bee-6b0ddfd11b10;1.0
	confs: [default]
	found org.apache.spark#spark-hadoop-cloud_2.13;4.0.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.hadoop#hadoop-aws;3.4.1 in central
	found org.wildfly.openssl#wildfly-openssl;2.2.5.Final in central
	found s

In [4]:
df = spark.read.option("header","true").csv("s3a://bronze/flights/")
df.show(5, truncate=False)

26/01/11 23:36:47 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
26/01/11 23:36:58 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://bronze/flights/.
java.io.FileNotFoundException: No such file or directory: s3a://bronze/flights
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:4156)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:4007)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$getFileStatus$22(S3AFileSystem.java:3984)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.invokeTrackingDuration(IOStatisticsBinding.java:547)
	at org.apache.hadoop.fs.statistics.impl.IO

AnalysisException: [PATH_NOT_FOUND] Path does not exist: s3a://bronze/flights. SQLSTATE: 42K03

In [6]:
from pyspark.sql.functions import current_date
df_silver = df.withColumn("ingestion_date", current_date())

In [7]:
silver_out = "s3a://silver/flights_delta"

In [8]:
df_silver.write.format("delta").mode("overwrite").save(silver_out)

In [9]:
df_delta = spark.read.format("delta").load(silver_out)

In [10]:
df_delta.show()

26/01/11 13:34:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+--------+-----+--------+------+-----------+--------------+
|    date|delay|distance|origin|destination|ingestion_date|
+--------+-----+--------+------+-----------+--------------+
|02151800|  108|     290|   ORD|        MSP|    2026-01-11|
|02151800|  142|     772|   ORD|        DEN|    2026-01-11|
|02151303|   16|    1516|   ORD|        LAX|    2026-01-11|
|02151157|    7|    1316|   ORD|        LAS|    2026-01-11|
|02151818|   55|    1511|   ORD|        PDX|    2026-01-11|
|02151033|   12|     873|   ORD|        MCO|    2026-01-11|
|02150941|    0|    1499|   ORD|        SNA|    2026-01-11|
|02151320|   17|    1604|   ORD|        SFO|    2026-01-11|
|02151804|    2|    1497|   ORD|        SAN|    2026-01-11|
|02152000|   17|     119|   ORD|        GRR|    2026-01-11|
|02151502|   15|     260|   ORD|        DSM|    2026-01-11|
|02151935|   73|     879|   ORD|        TPA|    2026-01-11|
|02151315|   -1|     879|   ORD|        TPA|    2026-01-11|
|02150957|    0|     939|   ORD|        

In [11]:
spark.stop()